# Task 06 👨🏻‍💻

Team Task

Task Description 📄

❄️ Create a program that perform below mentioned task upon recognizing a particular face. 

📌 When it recognize your face then - <br>
👉 It send mail to your mail id by writing this is face of your_name. <br>
👉 Second it send whatsapp message to your friend, it can be anything.<br> 
👉 Create EC2 instance in the AWS using CLI. <br>
👉 Create 5 GB EBS volume and attach it to the instance. 











...

## Initial setup requirement

1 create Security group using cli:

$ aws ec2 create-security-group --group-name my-security-group --description "My first security group using AWS CLI"

2 create a key pair using cli or manully create through web UI:

$ aws ec2 create-key-pair --key-name ritesh_cli_kpn

In [1]:
import pywhatkit
import smtplib
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import datetime
import sys

In [2]:
import subprocess as sp
import json, re, time

def createAWSInstance():
    out=sp.getstatusoutput("aws --version")
    print(out)

    #Initial Setup requirements:
    security_group_id="sg-0598cc60665e53f5f"
    keyPair_name='ritesh_cli_kpn'
    
    start a new instance
    ec2=sp.getstatusoutput("aws ec2 run-instances --image-id ami-0ad704c126371a549 --instance-type t2.micro --count 1 --key-name "+keyPair_name+"  --security-group-id  "+ security_group_id)

    #start a stopped instance with id
    #ec2=sp.getstatusoutput("aws ec2 start-instances --instance-ids i-0978d043e70")
    
    print(ec2[1])
    
    pattern = r'\n'
    # Remove characters '\n' from a string ec2[1]
    mod_string = re.sub(pattern, '', ec2[1])
    ec2_dict=json.loads(mod_string)
    print(ec2_dict)
    ec2_id=ec2_dict["Instances"][0]['InstanceId']
    print(ec2_id)
    
    ec2_zone=ec2_dict["Instances"][0]["Placement"]['AvailabilityZone']
    print(ec2_zone)
    
    #create a new volume of size 1gb 
    new_volume=sp.getstatusoutput("aws ec2 create-volume --availability-zone "+ec2_zone+" --size 5 --volume-type gp2")
    print(new_volume[1])
    
    pattern = r'\n'
    # Remove characters '\n' from a string ec2[1]
    mod_string = re.sub(pattern, '', new_volume[1])
    #print(mod_string)
    vol_dict=json.loads(mod_string)
    print(vol_dict)
    vol_id=vol_dict["VolumeId"]
    print(vol_id)
    
    time.sleep(20)
    #attach the newly created volume to ec2 instance
    cmnd="aws ec2 attach-volume --instance-id "+ec2_id+" --volume-id "+vol_id+" --device /dev/xvdh"
    stat=sp.getstatusoutput(cmnd)
    print(stat[1])
    

## Capture Images

In [3]:


face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

print(face_classifier)

def face_extractor(img):

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if faces is ():
        return None

    for (x, y, w, h) in faces:
        cropped_face = img[y:y + h, x:x + w]

    return cropped_face


cap = cv2.VideoCapture(0)
count = 0

while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = r'C:\Users\Ritesh\Desktop\Faces\\'
        cv2.imwrite(file_name_path+str(count)+r'ab.jpg', face)

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Face Cropper', face)

    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100:  # 13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()




<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-b5fa07e9c51e>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


<CascadeClassifier 000001AA7B362690>


## Train Model

In [4]:
data_path = r"C:\Users\Ritesh\Desktop\Faces\\"
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

#print(onlyfiles)
Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

Labels = np.asarray(Labels, dtype=np.int32)

#print(dir(cv2))
our_model = cv2.face.LBPHFaceRecognizer_create()

our_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained successfully")

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 2)
        roi = img[y:y + h, x:x + w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

cap = cv2.VideoCapture(0)

f=0
g=0
cnt=0
while g==0:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    if cnt<60:
        #print(cnt)
        print('.',end='')
        cnt+=1
        cv2.imshow('Face Recognition', image)
        if cv2.waitKey(1) == 13:  # 13 is the Enter Key
            break
        continue
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = our_model.predict(face)
        print(results)
        if results[1] < 600:
            confidence = int(100 * (1 - (results[1]) / 400))
            print(confidence)
            display_string = str(confidence) + '% Confident it is User'
            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 120, 150), 2)
            cv2.imshow('Face Recognition', image)
            if confidence > 90 and g==0:
                g=1
                #send mail Using SMTP
                server = smtplib.SMTP('smtp.gmail.com', 587)
                server.starttls()
                server.login('abc@gmail.com', 'rzbc3')
                server.sendmail('abc@gmail.com', 'rcw@gmail.com', 'This is the face of Ritesh')
                print("mail sent")
                
                # send Whatsapp Message
                
                now = datetime.datetime.now()
                print(now)
                pywhatkit.sendwhatmsg('+911122233344', 'Hello From Windows Machine!!! This is system generated, Ignore it', now.hour, now.minute+1)
                print("WhatsApp msg sent")
                
                #Create an AWS instance and attach volume to it
                if f==0:
                    f=1
                    createAWSInstance()
                sys.exit()
                break
        else:

            cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow('Face Recognition', image)

    except:
        #cv2.putText(image, "No Face Found", (220, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('Face Recognition', image)
        

    if cv2.waitKey(1) == 13:  # 13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()


<>:26: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:26: SyntaxWarning: "is" with a literal. Did you mean "=="?


Model trained successfully


<ipython-input-4-0e773709f545>:26: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


............................................................(55, 22.806001351846646)
94
mail sent
2021-06-25 23:03:25.667766
In 15 seconds web.whatsapp.com will open and after 20 seconds message will be delivered
WhatsApp msg sent
(0, 'aws-cli/2.2.13 Python/3.8.8 Windows/10 exe/AMD64 prompt/off')
{
    "StartingInstances": [
        {
            "CurrentState": {
                "Code": 0,
                "Name": "pending"
            },
            "InstanceId": "i-0978d043e1d3d6770",
            "PreviousState": {
                "Code": 80,
                "Name": "stopped"
            }
        }
    ]
}
{
    "AvailabilityZone": "ap-south-1b",
    "CreateTime": "2021-06-25T17:34:10+00:00",
    "Encrypted": false,
    "Size": 5,
    "SnapshotId": "",
    "State": "creating",
    "VolumeId": "vol-0cc86cc505e93851f",
    "Iops": 100,
    "Tags": [],
    "VolumeType": "gp2",
    "MultiAttachEnabled": false
}
{'AvailabilityZone': 'ap-south-1b', 'CreateTime': '2021-06-25T17:34:10+00:00